# 第10章：決定木

## 10.2 決定木モデルの実装

### 10.2.1 モジュールなどの宣言

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split # ホールドアウト用モジュール
from sklearn.tree import DecisionTreeClassifier as dtc # 決定木用モジュール
from sklearn.tree import plot_tree # 決定木の可視化モジュール
from sklearn.metrics import accuracy_score, roc_auc_score # 評価指標用モジュール

### 10.2.2 pickleファイルの読み込み

In [ ]:
# 前処理済み中間データのdictを読み取る
pp_data_dict = joblib.load("../intermediate/pp_data_dict_pkl.pkl3")

# 辞書型変数の値に格納された決定木向け中間データを読み取る
dtc_train_df = pp_data_dict["dtc"]["train"]
dtc_test_df = pp_data_dict["dtc"]["test"]

### 10.2.3 訓練データの分割

In [ ]:
# 6:4の割合でホールドアウト法を行う
dtc_train_train_df, dtc_train_valid_df = train_test_split(dtc_train_df, test_size=0.4, random_state=57, shuffle=True)

In [ ]:
dtc_train_df.shape, dtc_train_train_df.shape, dtc_train_valid_df.shape, dtc_test_df.shape

### 10.2.4 モデルの構築

In [ ]:
# 目的変数をtargetという変数に格納する
target = "buy_flag"

# 説明変数をfeaturesという変数に格納する
dtc_features = dtc_train_df.columns.tolist()

# customer_idとbuy_flagは説明変数ではない為削除する
dtc_features.remove("customer_id")
dtc_features.remove("buy_flag")

In [ ]:
# 定義する
dtc_clf = dtc(max_depth=3, random_state=57)

In [ ]:
# 学習する
dtc_clf.fit(dtc_train_train_df[dtc_features], dtc_train_train_df[target])

In [ ]:
# 予測する（predict_proba）
train_valid_proba_y = dtc_clf.predict_proba(dtc_train_valid_df[dtc_features]).T[1]
train_valid_proba_y

In [ ]:
# 検証する
# AUCを出力する
train_valid_auc_val = roc_auc_score(dtc_train_valid_df[target], train_valid_proba_y)
train_valid_auc_val

In [ ]:
# 予測する（predict）
train_valid_pred_y = dtc_clf.predict(dtc_train_valid_df[dtc_features])

# Accuracyを出力する
train_valid_accuracy_val = accuracy_score(dtc_train_valid_df[target], train_valid_pred_y)
train_valid_accuracy_val

### 10.2.5 決定木の可視化

In [ ]:
# 決定木の可視化を行う
plt.figure(figsize=(20, 10), facecolor="white", dpi=150)
plot_tree(dtc_clf, feature_names=dtc_features, class_names=["not_buy", "buy"], fontsize=8, filled=True)
plt.show()

## 10.3 ハイパーパラメータチューニング

### 10.3.3 グリッドサーチ

In [ ]:
# ハイパーパラメータチューニング
# グリッドサーチ
# 探索するハイパーパラメータの候補をリストに格納する.
dtc_params_list = [{"max_depth": ii, "random_state": 57} for ii in np.arange(2, 12, 1)]
dtc_params_list

In [ ]:
# グリッドサーチの結果を格納するリスト
gs_result_list = []

# グリッドサーチの実施
for params_dict in dtc_params_list:
    # 定義する
    tmp_dtc_clf = dtc(**params_dict)
    # 学習する
    tmp_dtc_clf.fit(dtc_train_train_df[dtc_features], dtc_train_train_df[target])
    
    # train-trainを予測する（predict関数）
    train_pred_y = tmp_dtc_clf.predict(dtc_train_train_df[dtc_features])
    
    # train-trainを予測する（predict_proba関数）
    train_proba_y = tmp_dtc_clf.predict_proba(dtc_train_train_df[dtc_features]).T[1]
    
    # train-validを予測する（predict関数）
    valid_pred_y = tmp_dtc_clf.predict(dtc_train_valid_df[dtc_features])
    
    # train-validを予測する（predict_proba関数）
    valid_proba_y = tmp_dtc_clf.predict_proba(dtc_train_valid_df[dtc_features]).T[1]
    
    # train-trainを検証する
    train_accuracy_val = accuracy_score(dtc_train_train_df[target], train_pred_y)
    train_auc_val = roc_auc_score(dtc_train_train_df[target], train_proba_y)
    
    # train-validを検証する
    valid_accuracy_val = accuracy_score(dtc_train_valid_df[target], valid_pred_y)
    valid_auc_val = roc_auc_score(dtc_train_valid_df[target], valid_proba_y)
    
    # リストに格納する
    gs_result_list += [[params_dict, params_dict["max_depth"], train_accuracy_val, train_auc_val, valid_accuracy_val, valid_auc_val]]

In [ ]:
# 結果をDataFrameに格納する
gs_result_df = pd.DataFrame(gs_result_list, columns=["params_dict", "max_depth", "train_accuracy", "train_auc", "valid_accuracy", "valid_auc"])

# 結果を確認する
gs_result_df[["max_depth", "train_accuracy", "train_auc", "valid_accuracy", "valid_auc"]].sort_values(by="valid_auc", ascending=False).reset_index(drop=True)

In [ ]:
# Accuracyの可視化
plt.figure(figsize=(6, 5), facecolor="white", dpi=150)
# train Accuracy
plt.plot(gs_result_df["max_depth"], gs_result_df["train_accuracy"], label="train_accuracy")
# valid Accuracy
plt.plot(gs_result_df["max_depth"], gs_result_df["valid_accuracy"], label="valid_accuracy")
plt.title("Accuracy\nDTC max_depth tuning", fontsize=10)
plt.xlabel("max_depth", fontsize=10)
plt.ylabel("Accuracy", fontsize=10)
plt.legend()
plt.grid()
plt.show()

In [ ]:
# AUCの可視化
plt.figure(figsize=(6, 5), facecolor="white", dpi=150)
# train AUC
plt.plot(gs_result_df["max_depth"], gs_result_df["train_auc"], label="train_auc")
# valid AUC
plt.plot(gs_result_df["max_depth"], gs_result_df["valid_auc"], label="valid_auc")
plt.title("AUC\nDTC max_depth tuning", fontsize=10)
plt.xlabel("max_depth", fontsize=10)
plt.ylabel("AUC", fontsize=10)
plt.legend()
plt.grid()
plt.show()

## 10.4 最適化した決定木モデルの実装

### 10.4.1 モデルの構築

In [ ]:
# train-validのAUCが最も高いハイパーパラメータの組み合わせを変数に格納する
dtc_best_idx = np.argmax(gs_result_df["valid_auc"])
dtc_best_params = gs_result_df["params_dict"].values[dtc_best_idx]
dtc_best_score = gs_result_df["valid_auc"].values[dtc_best_idx]

# 内容を確認
print("dtc_best_params:", dtc_best_params)
print("dtc_best_score:", dtc_best_score)

In [ ]:
# 最適化したハイパーパラメータを使用してモデルを定義する
dtc_clf = dtc(**dtc_best_params)

# 学習する
dtc_clf.fit(dtc_train_df[dtc_features], dtc_train_df[target])

In [ ]:
# 決定木のインポータンスを確認する
importance_df = pd.DataFrame({"feature": dtc_features, "importance": dtc_clf.feature_importances_})
importance_df = importance_df.sort_values(by="importance", ascending=False).reset_index(drop=True)
importance_df.head()

### 10.4.2 モデルの可視化

In [ ]:
# 決定木の可視化を行う
plt.figure(figsize=(20, 10), facecolor="white", dpi=150)
plot_tree(dtc_clf, feature_names=dtc_features, class_names=["not_buy", "buy"], fontsize=8, filled=True)
plt.show()

### 10.4.3 予測と検証

In [ ]:
# trainを予測する
train_pred_y = dtc_clf.predict(dtc_train_df[dtc_features])
train_proba_y = dtc_clf.predict_proba(dtc_train_df[dtc_features]).T[1]

# trainを検証する
train_accuracy_val = accuracy_score(dtc_train_df[target], train_pred_y)
train_auc_val = roc_auc_score(dtc_train_df[target], train_proba_y)
train_accuracy_val, train_auc_val

### 10.4.4 テストデータの予測

In [ ]:
# testを予測する
test_pred_y = dtc_clf.predict(dtc_test_df[dtc_features])
test_proba_y = dtc_clf.predict_proba(dtc_test_df[dtc_features]).T[1]

In [ ]:
# sample submitデータを読み込む
gi_sample_submit_df = pd.read_csv("../input/gi_sample_submit.csv")

In [ ]:
# submit向けDataFrameを作成し、列に予測確率を格納する
submit_df = dtc_test_df.copy()[["customer_id"]]
submit_df["buy_proba"] = test_proba_y
submit_df.head()

In [ ]:
gi_sample_submit_df.shape, submit_df.shape

In [ ]:
submit_df = pd.merge(gi_sample_submit_df.drop("buy_proba", axis=1), submit_df, on="customer_id", how="left").reset_index(drop=True)
submit_df.head()

In [ ]:
submit_df.shape

In [ ]:
# outputディレクトリにsubmit用ファイルを出力する
submit_df.to_csv(f"../output/submit_dtc.csv", encoding="utf-8", index=False)